In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from PIL import Image
import cv2
from gender_config import *

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Create the tf.data.dataset

In [4]:
def load_utk(file_path):
  # obtain label
  label = tf.strings.split(file_path, os.path.sep)[-2]
  label = tf.strings.to_number(label, out_type=tf.dtypes.int32)
  # label = tf.one_hot(label, 2) # for one hot encoding
  label = tf.cast(label, dtype=tf.uint8)

  # obtain image
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, IMG_SIZE)
  img = img/255

  return img, label

In [5]:
def image_augmentations(image, label):
  image = tf.image.random_saturation(image, 0.7, 1.3)
  image = tf.image.random_hue(image, 0.05)
  image = tf.image.random_brightness(image, 0.2)
  image = tf.image.adjust_gamma(image, gamma=0.2, gain=0.2)
  image = tf.image.random_flip_left_right(image)

  return image, label

Load in the data

In [6]:
utk_ds = tf.data.Dataset.list_files(GENDER_DATA + '/*/*')

Split into train/val

In [7]:
data_len = len(utk_ds)
# shuffle data for splitting
utk_ds = utk_ds.shuffle(data_len, reshuffle_each_iteration=True)

# split data
val_len = int(data_len * VAL_SPLIT)
train_ds = utk_ds.skip(val_len)
val_ds = utk_ds.take(val_len)

Data Pipeline

In [8]:
train_ds = train_ds.map(load_utk, num_parallel_calls=tf.data.AUTOTUNE).map(image_augmentations, num_parallel_calls=tf.data.AUTOTUNE).cache().batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(load_utk, num_parallel_calls=tf.data.AUTOTUNE).cache().batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

## Create, Train Model

In [12]:
model = keras.models.Sequential([
    layers.Conv2D(32, (2, 2), padding = "same", activation='relu', input_shape=IMG_SHAPE),
    keras.layers.Dropout(0.2), 
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (2, 2), padding = "same", activation='relu'),
    keras.layers.Dropout(0.2), 
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (2,2), 2, activation='relu'),
    keras.layers.Dropout(0.2), 
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), 2, activation='relu'),
    # layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    # keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2), 
    # keras.layers.Dense(128, activation="relu"),
    # keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),    
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss='binary_crossentropy', 
                optimizer="Adam", 
                metrics=['binary_accuracy'])

In [14]:
callbacks = [
        # tf.keras.callbacks.TensorBoard(log_dir=LOG_PATH, histogram_freq=1),
        tf.keras.callbacks.ModelCheckpoint(MODEL_PATH,
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='min')]

model.fit(train_ds, epochs=20, verbose = 1, callbacks = callbacks,
         validation_data = val_ds)

Epoch 1/20
314/314 [==============================] - 34s 106ms/step - loss: 0.5232 - binary_accuracy: 0.7305 - val_loss: 0.3888 - val_binary_accuracy: 0.8304

Epoch 00001: val_loss improved from inf to 0.38884, saving model to ./models\val_loss_0.389.hdf5
Epoch 2/20
314/314 [==============================] - 32s 102ms/step - loss: 0.3516 - binary_accuracy: 0.8454 - val_loss: 0.3352 - val_binary_accuracy: 0.8560

Epoch 00002: val_loss improved from 0.38884 to 0.33519, saving model to ./models\val_loss_0.335.hdf5
Epoch 3/20
314/314 [==============================] - 31s 98ms/step - loss: 0.3160 - binary_accuracy: 0.8614 - val_loss: 0.3030 - val_binary_accuracy: 0.8747

Epoch 00003: val_loss improved from 0.33519 to 0.30302, saving model to ./models\val_loss_0.303.hdf5
Epoch 4/20
314/314 [==============================] - 31s 99ms/step - loss: 0.2927 - binary_accuracy: 0.8753 - val_loss: 0.2770 - val_binary_accuracy: 0.8832

Epoch 00004: val_loss improved from 0.30302 to 0.27703, saving 

In [15]:
model.save('gender_model.hdf5')